In [134]:
#Import needed modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from matplotlib import pyplot as plt

In [37]:
#Load the data
df = pd.read_csv("bank-additional.csv", header= 'infer', sep = ';')

In [38]:
#Inspect the top 10 rows of data
df.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
5,32,services,single,university.degree,no,no,no,cellular,sep,thu,...,3,999,2,failure,-1.1,94.199,-37.5,0.884,4963.6,no
6,32,admin.,single,university.degree,no,yes,no,cellular,sep,mon,...,4,999,0,nonexistent,-1.1,94.199,-37.5,0.879,4963.6,no
7,41,entrepreneur,married,university.degree,unknown,yes,no,cellular,nov,mon,...,2,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no
8,31,services,divorced,professional.course,no,no,no,cellular,nov,tue,...,1,999,1,failure,-0.1,93.200,-42.0,4.153,5195.8,no
9,35,blue-collar,married,basic.9y,unknown,no,no,telephone,may,thu,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no


In [39]:
#Convert the y column to a bit operator, indiactes whether or not customer took out a savings bond
df.loc[df['y'] == 'yes', 'sale'] = 1
df.loc[df['y'] == 'no', 'sale'] = 0

In [40]:
#One hot encoding goes here, we MUST one hot encode 
df = df.drop('y', axis =1)
df_ohe = pd.get_dummies(df)

In [42]:
#Create our input values matrix (X) and our outcome variable (y)
X = df_ohe.drop('sale', axis = 1)
y = df_ohe['sale'].values #could also just use y from original data frame

In [43]:
#Split into training and test sets, 70:30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [62]:
#Build fit and return the predictions from our initial random forest estimator
regressor = RandomForestRegressor(n_estimators=20, random_state=123)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [186]:
print('AUC:', metrics.roc_auc_score(y_test, y_pred))

AUC: 0.88933690437649


In [151]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 2000, num = 100)]

# Minimum number of samples required to split a node
min_samples_split = [20, 30, 40, 50]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8, 16]

# Create the random grid
param_grid = {'n_estimators': n_estimators
              #,'min_samples_split': min_samples_split
              #, 'min_samples_leaf' : min_samples_leaf
             }
# Create scoring matrix

#scoring = {'AUC': 'roc_auc'}#, 'Accuracy': make_scorer(accuracy_score)}

In [158]:
rf_random = RandomizedSearchCV(estimator = regressor, 
                               param_distributions = param_grid, 
                               n_iter = 100, 
                               cv = 3, verbose=2, 
                               scoring = 'roc_auc',
                               random_state=123, n_jobs = -1)#, refit=False)
# Fit the random search model
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.5min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
           oob_score=False, random_state=123, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 520, 540, 560, 580, 600, 620, 640, 660, 680, 700, 720, 740, 760, 780, 800, 820, 840, 860, 880, 900, 920, 940, 960, 980, 1000, 1020, 1040, 106...940, 1960, 1980, 2000], 'min_samples_split': [20, 30, 40, 50], 'min_samples_leaf': [1, 2, 4, 8, 16]},
          pre_dispatch='2*n_jobs', random_state=123, refit=

In [188]:
print(rf_random.best_params_)
best = rf_random.best_params_
results = rf_random.cv_results_
print(rf_random.best_estimator_.score)

{'n_estimators': 1720, 'min_samples_split': 50, 'min_samples_leaf': 2}
<bound method RegressorMixin.score of RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=50,
           min_weight_fraction_leaf=0.0, n_estimators=1720, n_jobs=None,
           oob_score=False, random_state=123, verbose=0, warm_start=False)>


In [167]:
regressor2 = RandomForestRegressor(n_estimators=best['n_estimators']
                                   ,min_samples_split= best['min_samples_split']
                                   ,min_samples_leaf=best['min_samples_leaf']
                                   ,random_state=123)
regressor2.fit(X_train, y_train)
y_pred_2 = regressor2.predict(X_test)

In [170]:
print('AUC:', metrics.roc_auc_score(y_test, y_pred_2))

AUC: 0.9184227565706213


In [171]:
print('AUC:', metrics.roc_auc_score(y_test, y_pred))

AUC: 0.88933690437649
